In [7]:
import os
import requests
import csv


gemeinden = []
with open('../data/vese/Gemeindestand.csv', 'r') as file:
    reader = csv.reader(file, delimiter=';')  # Verwende ; als Trennzeichen
    next(reader)  # Überspringe die Kopfzeile
    for row in reader:
        gemeinden.append(row[4])

# Lizenzschlüssel (API-Key)
license_key = 'vkmshstyf3r9epknhajqhenzog4opv6wcajd7ob7'

# Funktion zur Abfrage der Gemeinde-API
def abfrage_gemeinde(idofs, license_key):
    url = f'https://opendata.vese.ch/pvtarif/api/getData/muni?idofs={idofs}&licenseKey={license_key}'
    response = requests.get(url)
    return response.json()

# Ergebnisse speichern
ergebnisse = []

gemeinde_mit_fehler = []
for gemeinde in gemeinden:
    result = abfrage_gemeinde(gemeinde, license_key)
    if not result['valid']:
        gemeinde_mit_fehler.append(gemeinde)   
    else:
        for evu in result['evus']:
            ergebnisse.append([evu['nrElcom'], evu['Name'], evu['idofs'], evu['PLZ'], evu['Gemeinde']])

print("Gemeinden mit Fehlern:")
for gemeinde in gemeinde_mit_fehler:
    print(gemeinde)   

# Datei löschen, wenn sie existiert
output_file = '../data/vese/gemeinde_ergebnisse.csv'
if os.path.exists(output_file):
    os.remove(output_file)

# Ergebnisse in eine CSV-Datei schreiben
with open('../data/vese/gemeinde_ergebnisse.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file, delimiter=';')
    writer.writerow(['nrElcom', 'Name', 'idofs', 'PLZ', 'Gemeinde'])
    writer.writerows(ergebnisse)


Gemeinden mit Fehlern:
291
2238
2239
2430
3112
3396
3715
4185
4186
4324
5239
5240
5395
5399
5656
5892
6037
6254
6513
6811
6812
